# Convert the output of the stim to events.tsv in BIDS format
Task 02

In [81]:
import pandas
import os

In [ ]:
# Define important stuff
subID = "sub-20"
sesID = "ses-01"

taskListBIDS = ["task-02a_run-1", "task-02a_run-2", "task-02a_run-3", "task-02a_run-4"]

n_tasks = len(taskListBIDS)

raw_log_folder = os.path.join(
    os.path.dirname(os.getcwd()), "data", "logfiles_task02_stim"
)

#bids_folder = "/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2" # mac folder
bids_folder = "/DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2" # sim01 folder

In [83]:
# Define function to convert logfiles to TSV
def log2tsv(subID, sesID, taskID, raw_log_folder, bids_folder):
    tsv_name = f"output_{subID}_{taskID}.tsv"

    # read in the tsv file
    tsv_file = pandas.read_csv(os.path.join(raw_log_folder, tsv_name), sep="\t")

    # get the row number
    row_number = tsv_file.shape[0]

    # replace all 'Joyful activation' with 'JoyfulActivation', as the space leads to problems later on
    tsv_file["emotion"] = tsv_file["emotion"].replace(
        "Joyful activation", "JoyfulActivation"
    )

    # initalize dataframe with three columns: onset, duration, trial_type
    df = pandas.DataFrame(columns=["onset", "duration", "trial_type"])

    # start with the first noise block and add it to the dataframe (without using append)
    df.loc[0] = [0, round(tsv_file.iloc[0]["true_music_onset"], 2), "Noise_ToDiscard"]

    # iterator index for df
    idx = 1

    # then move on and read the tsv_file up to the before last row
    for i in range(row_number - 1):
        # get the row and the next row
        row = tsv_file.iloc[i]
        next_row = tsv_file.iloc[i + 1]

        # get the music onset, duration, and emotion
        onset_1 = round(row["true_music_onset"], 2)
        duration_1 = round(row["music_duration"], 2)
        trial_type_1 = row["emotion"]

        # get the noise onset, duration
        onset_2 = round(row["true_noise_onset"], 2)
        duration_2 = round(next_row["true_music_onset"] - row["true_noise_onset"], 2)

        # if the duration of the noise block is smaller than 10 seconds, we should call it Noise_InterSong
        if duration_2 < 10:
            trial_type_2 = "Noise_InterSong"
        else:
            trial_type_2 = "Noise"

        # append both to the dataframe
        df.loc[idx] = [onset_1, duration_1, trial_type_1]
        df.loc[idx + 1] = [onset_2, duration_2, trial_type_2]

        # update the iterator index
        idx += 2

    # now we have to add the last music and noise block
    last_row = tsv_file.iloc[-1]

    df.loc[idx] = [
        round(last_row["true_music_onset"], 2),
        round(last_row["music_duration"], 2),
        last_row["emotion"],
    ]
    df.loc[idx + 1] = [
        round(last_row["true_noise_onset"], 2),
        round(660 - last_row["true_noise_onset"], 2),
        "Noise",
    ]

    # save the dataframe as a tsv file to the BIDS directory
    df.to_csv(
        os.path.join(
            bids_folder, subID, sesID, "func", f"{subID}_{sesID}_{taskID}_events.tsv"
        ),
        sep="\t",
        index=False,
    )

    print(
        f"File exported: {os.path.join(bids_folder, subID, sesID, 'func', f'{subID}_{sesID}_{taskID}_events.tsv')}"
    )

In [84]:
# Iterate over all tasks
for taskID in taskListBIDS:
    log2tsv(subID, sesID, taskID, raw_log_folder, bids_folder)

File exported: /DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2/sub-01/ses-01/func/sub-01_ses-01_task-02a_run-1_events.tsv
File exported: /DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2/sub-01/ses-01/func/sub-01_ses-01_task-02a_run-2_events.tsv
File exported: /DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2/sub-01/ses-01/func/sub-01_ses-01_task-02a_run-3_events.tsv
File exported: /DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2/sub-01/ses-01/func/sub-01_ses-01_task-02a_run-4_events.tsv
